In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pygame
import sys
import copy
import random
import time
import psycopg2
from psycopg2 import sql

# --- DATABASE FUNCTIONS ---
def get_connection():
    return psycopg2.connect(
        dbname="snakegame",
        user="postgres",
        password="1234",
        host="localhost",
        port="5432"
    )

def init_db():
    """Initialize database tables if they don't exist"""
    try:
        conn = get_connection()
        cur = conn.cursor()
        
        # Create users table
        cur.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id SERIAL PRIMARY KEY,
            username VARCHAR(50) UNIQUE NOT NULL
        )""")
        
        # Create user_scores table (renamed from user_score for consistency)
        cur.execute("""
        CREATE TABLE IF NOT EXISTS user_scores (
            id SERIAL PRIMARY KEY,
            user_id INTEGER REFERENCES users(id),
            score INTEGER NOT NULL DEFAULT 0,
            level INTEGER NOT NULL DEFAULT 0,
            last_played TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )""")
        
        conn.commit()
        print("Database tables initialized successfully")
    except Exception as e:
        print(f"Database initialization error: {e}")
    finally:
        if conn:
            conn.close()

def get_or_create_user(username):
    """Get or create user and return their game state"""
    try:
        conn = get_connection()
        cur = conn.cursor()
        
        # Check if user exists
        cur.execute("SELECT id FROM users WHERE username = %s", (username,))
        user = cur.fetchone()
        
        if user:
            user_id = user[0]
            # Get user's game state
            cur.execute("""
                SELECT score, level FROM user_scores 
                WHERE user_id = %s 
                ORDER BY last_played DESC 
                LIMIT 1
            """, (user_id,))
            state = cur.fetchone()
            score, level = state if state else (0, 0)
            print(f"Welcome back, {username}! Current level: {level}")
        else:
            # Create new user
            cur.execute("""
                INSERT INTO users (username) 
                VALUES (%s) 
                RETURNING id
            """, (username,))
            user_id = cur.fetchone()[0]
            
            # Initialize game state
            cur.execute("""
                INSERT INTO user_scores (user_id, score, level) 
                VALUES (%s, 0, 0)
            """, (user_id,))
            score, level = 0, 0
            print(f"New user created: {username}")
        
        conn.commit()
        return user_id, score, level
        
    except Exception as e:
        print(f"Database error: {e}")
        return None, 0, 0
    finally:
        if conn:
            conn.close()

def save_user_state(user_id, score, level):
    """Save current game state to database"""
    try:
        conn = get_connection()
        cur = conn.cursor()
        
        cur.execute("""
            INSERT INTO user_scores (user_id, score, level)
            VALUES (%s, %s, %s)
        """, (user_id, score, level))
        
        conn.commit()
    except Exception as e:
        print(f"Error saving game state: {e}")
    finally:
        if conn:
            conn.close()

# --- GAME INITIALIZATION ---
pygame.init()
init_db()  # Ensure database tables exist

# Game configuration
SCALE = 15
WIDTH, HEIGHT = 500, 500
INITIAL_SPEED = 10

# Colors
BACKGROUND_TOP = (170, 51, 106)
BACKGROUND_BOTTOM = (0, 0, 0)
SNAKE_COLOR = (255, 137, 0)
SNAKE_HEAD = (255, 247, 0)
FOOD_COLOR = (255, 0, 0)
TEXT_COLOR = (255, 255, 255)
GAME_OVER_COLOR = (255, 0, 0)
WARNING_COLOR = (255, 50, 50)

# Setup display
display = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game with PostgreSQL")
clock = pygame.time.Clock()

# --- GAME CLASSES ---
class Snake:
    def __init__(self, x, y):
        self.size = SCALE
        self.x_dir = 1
        self.y_dir = 0
        self.length = 1
        self.history = [[x, y]]  # Snake body segments
        
    def move(self):
        # Move each segment to follow the head
        for i in range(self.length - 1, 0, -1):
            self.history[i] = copy.deepcopy(self.history[i - 1])
        
        # Move head based on direction
        self.history[0][0] += self.x_dir * SCALE
        self.history[0][1] += self.y_dir * SCALE
        
        # Screen wrapping
        self.history[0][0] %= WIDTH
        self.history[0][1] %= HEIGHT
    
    def grow(self):
        self.length += 1
        self.history.append(copy.deepcopy(self.history[-1]))
    
    def check_collision(self):
        # Check if head collides with body
        head = self.history[0]
        return any(segment == head for segment in self.history[1:])
    
    def draw(self):
        for i, segment in enumerate(self.history):
            color = SNAKE_HEAD if i == 0 else SNAKE_COLOR
            pygame.draw.rect(display, color, (*segment, self.size, self.size))
    
    def reset(self):
        self.__init__(WIDTH // 2, HEIGHT // 2)

class Food:
    def __init__(self):
        self.size = SCALE
        self.spawn_time = time.time()
        self.respawn()
    
    def respawn(self):
        self.x = random.randint(0, (WIDTH - SCALE) // SCALE) * SCALE
        self.y = random.randint(0, (HEIGHT - SCALE) // SCALE) * SCALE
        self.weight = random.randint(1, 5)
        self.spawn_time = time.time()
        self.color = (
            min(255, 50 + self.weight * 40),
            max(0, 255 - self.weight * 40),
            random.randint(50, 200)
        )
    
    def time_left(self):
        return max(0, 5 - (time.time() - self.spawn_time))
    
    def draw(self):
        # Flash red when about to disappear
        if self.time_left() < 3 and int(time.time() * 2) % 2 == 0:
            color = WARNING_COLOR
        else:
            color = self.color
            
        pygame.draw.rect(display, color, (self.x, self.y, self.size, self.size))
        
        # Draw weight number
        font = pygame.font.SysFont(None, 20)
        text = font.render(str(self.weight), True, TEXT_COLOR)
        display.blit(text, (self.x + 5, self.y + 5))

# --- GAME FUNCTIONS ---
def draw_background():
    """Draw gradient background"""
    for y in range(HEIGHT):
        r = BACKGROUND_TOP[0] + (BACKGROUND_BOTTOM[0] - BACKGROUND_TOP[0]) * y / HEIGHT
        g = BACKGROUND_TOP[1] + (BACKGROUND_BOTTOM[1] - BACKGROUND_TOP[1]) * y / HEIGHT
        b = BACKGROUND_TOP[2] + (BACKGROUND_BOTTOM[2] - BACKGROUND_TOP[2]) * y / HEIGHT
        pygame.draw.line(display, (r, g, b), (0, y), (WIDTH, y))

def show_game_info(score, level, food_time_left):
    """Display score, level and food timer"""
    font = pygame.font.SysFont(None, 25)
    
    score_text = font.render(f"Score: {score}", True, TEXT_COLOR)
    level_text = font.render(f"Level: {level}", True, TEXT_COLOR)
    time_text = font.render(f"Food: {int(food_time_left)}s", True, TEXT_COLOR)
    
    display.blit(score_text, (10, 10))
    display.blit(level_text, (120, 10))
    display.blit(time_text, (230, 10))

def pause_game(user_id, score, level):
    """Pause game and show pause menu"""
    save_user_state(user_id, score, level)
    paused = True
    font = pygame.font.SysFont(None, 60)
    
    while paused:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_p:
                    paused = False
                elif event.key == pygame.K_q:
                    pygame.quit()
                    sys.exit()
        
        display.fill((0, 0, 0))
        display.blit(font.render("PAUSED", True, TEXT_COLOR), (160, 200))
        pygame.display.update()
        clock.tick(5)

# --- MAIN GAME LOOP ---
def game_loop(user_id):
    global display, clock
    
    snake = Snake(WIDTH // 2, HEIGHT // 2)
    food = Food()
    score, level = 0, 0
    speed = INITIAL_SPEED
    
    running = True
    while running:
        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                save_user_state(user_id, score, level)
                running = False
            
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    save_user_state(user_id, score, level)
                    running = False
                elif event.key == pygame.K_p:
                    pause_game(user_id, score, level)
                # Direction controls (no 180° turns)
                elif event.key == pygame.K_UP and snake.y_dir == 0:
                    snake.x_dir, snake.y_dir = 0, -1
                elif event.key == pygame.K_DOWN and snake.y_dir == 0:
                    snake.x_dir, snake.y_dir = 0, 1
                elif event.key == pygame.K_LEFT and snake.x_dir == 0:
                    snake.x_dir, snake.y_dir = -1, 0
                elif event.key == pygame.K_RIGHT and snake.x_dir == 0:
                    snake.x_dir, snake.y_dir = 1, 0
        
        # Game logic
        snake.move()
        
        # Check if food was eaten
        if (abs(snake.history[0][0] - food.x) < SCALE and 
            abs(snake.history[0][1] - food.y) < SCALE):
            score += food.weight
            snake.grow()
            food.respawn()
            
            # Level up every 5 points
            if score // 5 > level:
                level = score // 5
                speed += 1
        
        # Check if food expired
        if food.time_left() <= 0:
            food.respawn()
        
        # Check for collisions
        if snake.check_collision():
            save_user_state(user_id, score, level)
            
            # Game over screen
            font = pygame.font.SysFont(None, 72)
            display.blit(font.render("GAME OVER", True, GAME_OVER_COLOR), (100, 200))
            pygame.display.update()
            time.sleep(2)
            
            # Reset game
            score, level = 0, 0
            speed = INITIAL_SPEED
            snake.reset()
            food.respawn()
        
        # Drawing
        draw_background()
        snake.draw()
        food.draw()
        show_game_info(score, level, food.time_left())
        
        pygame.display.update()
        clock.tick(speed)

# --- START THE GAME ---
if __name__ == "__main__":
    username = input("Enter your username: ").strip()
    if not username:
        print("Username cannot be empty!")
        sys.exit()
    
    user_id, score, level = get_or_create_user(username)
    if user_id is not None:
        game_loop(user_id)
    
    pygame.quit()
    sys.exit()